# Übernahme der Daten aus OSM Protobuf, da hier alle Tages enthalten sind

In [7]:
import duckdb
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests

## Url zurGeofabrik und lokale Ablage (zur Beschleunigung)

In [8]:
url = 'https://download.geofabrik.de/europe/germany/niedersachsen-latest.osm.pbf'
output_path = "downloads/niedersachsen-latest.osm.pbf"

In [ ]:
# Download a PDF file to the downloads folder
response = requests.get(url)
with open(output_path, "wb") as f:
    f.write(response.content)
print(f"Downloaded file to {output_path}")

Downloaded file to downloads/niedersachsen-latest.osm.pbf


In [9]:
duck = duckdb.connect(database=':memory:')

In [10]:
duck.sql("""INSTALL spatial;
LOAD spatial;""")

In [11]:
duck.sql(f"DESCRIBE SELECT * FROM ST_READOSM('{output_path}');")

┌─────────────┬──────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │                 column_type                  │  null   │   key   │ default │  extra  │
│   varchar   │                   varchar                    │ varchar │ varchar │ varchar │ varchar │
├─────────────┼──────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ kind        │ ENUM('node', 'way', 'relation', 'changeset') │ YES     │ NULL    │ NULL    │ NULL    │
│ id          │ BIGINT                                       │ YES     │ NULL    │ NULL    │ NULL    │
│ tags        │ MAP(VARCHAR, VARCHAR)                        │ YES     │ NULL    │ NULL    │ NULL    │
│ refs        │ BIGINT[]                                     │ YES     │ NULL    │ NULL    │ NULL    │
│ lat         │ DOUBLE                                       │ YES     │ NULL    │ NULL    │ NULL    │
│ lon         │ DOUBLE                                       │ YES     │ 

In [14]:
duck.sql(f"""select 
         tags['name'], tags['highway'], tags['maxspeed'], tags['oneway'], tags['oneway:bus'], tags['area'],  tags['access'], refs, *
        
         from ST_READOSM('{output_path}')
          where tags['highway'] is not null and kind = 'way'
          LIMIT 500;         
         """)

┌─────────────────────┬─────────────────┬──────────────────┬────────────────┬────────────────────┬──────────────┬────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────┬─────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────┬────────┬───────────┬────────

In [50]:
duck.sql("""
         CREATE TEMP TABLE matching_ways_with_nodes_refs AS
SELECT id, UNNEST(refs) as ref, UNNEST(range(length(refs))) as ref_idx
FROM ST_READOSM('monaco-latest.osm.pbf')
SEMI JOIN matching_ways USING (id)
WHERE kind = 'way';

SELECT * FROM matching_ways_with_nodes_refs;
         """)

CatalogException: Catalog Error: Table with name matching_ways does not exist!
Did you mean "pg_settings"?

LINE 5: SEMI JOIN matching_ways USING (id)
                  ^